In [1]:
from keras.applications.vgg16 import VGG16
import keras
from keras.models import Model
from keras.layers import Dense,GlobalAveragePooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.
/home/jiahao/anaconda3/envs/tensorflow_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jiahao/anaconda3/envs/tensorflow_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jiahao/anaconda3/envs/tensorflow_gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

In [2]:
vggmodel = VGG16(weights='imagenet', include_top=False)

In [5]:
trdata = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input)

traindata = trdata.flow_from_directory(directory="datasets/train",target_size=(224,224),color_mode='rgb')
# trdata.fit(traindata)

tsdata = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input)

testdata = tsdata.flow_from_directory(directory="datasets/val", target_size=(224,224),color_mode='rgb')

Found 65918 images belonging to 45 classes.
Found 16499 images belonging to 45 classes.


In [ ]:
# traindata.classes.

In [ ]:
# vggmodel.summary()

num_class = np.unique(testdata.classes).shape[0]


x=vggmodel.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3

preds = Dense(num_class, activation="softmax")(x)
model_final = Model(inputs = vggmodel.input, outputs = preds)




for layer in model_final.layers[:20]:
    layer.trainable=False
for layer in model_final.layers[20:]:
    layer.trainable=True
    
    

    
model_final.compile(loss = "categorical_crossentropy", 
                    optimizer = 'Adam', metrics=["accuracy","categorical_accuracy"])


checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1,
                             save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')


model_final.fit_generator(generator= traindata, steps_per_epoch= 2065, epochs= 100,
                          validation_data= testdata, validation_steps=0, callbacks=[checkpoint,early])

model_final.save_weights("vgg16_1.h5")

Epoch 1/100
2065/2065 [==============================] - 813s 393ms/step - loss: 3.0028 - acc: 0.2191 - categorical_accuracy: 0.2191 - val_loss: 2.8181 - val_acc: 0.2540 - val_categorical_accuracy: 0.2540

Epoch 00001: val_acc improved from -inf to 0.25402, saving model to vgg16_1.h5
Epoch 2/100
2065/2065 [==============================] - 837s 405ms/step - loss: 2.7304 - acc: 0.2679 - categorical_accuracy: 0.2679 - val_loss: 2.6943 - val_acc: 0.2809 - val_categorical_accuracy: 0.2809

Epoch 00002: val_acc improved from 0.25402 to 0.28087, saving model to vgg16_1.h5
Epoch 3/100
2065/2065 [==============================] - 834s 404ms/step - loss: 2.6254 - acc: 0.2877 - categorical_accuracy: 0.2877 - val_loss: 2.6018 - val_acc: 0.2979 - val_categorical_accuracy: 0.2979

Epoch 00003: val_acc improved from 0.28087 to 0.29790, saving model to vgg16_1.h5
Epoch 4/100
2065/2065 [==============================] - 833s 403ms/step - loss: 2.5476 - acc: 0.3052 - categorical_accuracy: 0.3052 - val_

/home/jiahao/anaconda3/envs/tensorflow_gpu/lib/python3.6/site-packages/keras/utils/data_utils.py:610: UserWarning: The input 1787 could not be retrieved. It could be because a worker has died.
  UserWarning)


In [ ]:
model_final.summary()

In [ ]:
# vggmodel.summary()

In [ ]:
num_class = np.unique(testdata.classes).shape[0]